In [1]:
import json, folium, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

In [3]:
park = pd.read_csv('data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
park = park[['공원명','소재지지번주소','공원면적']]
park.dropna(how='any', inplace=True)
park.head()

,공원명,소재지지번주소,공원면적
0,녹색소공원,서울특별시 용산구 효창동 287-6,1569.9
23,강변공원,서울특별시 용산구 이촌동 408,1766.0
24,솔밭어린이공원,서울특별시 용산구 한강로1가 30-3,2090.0
25,신창동어린이공원,서울특별시 용산구 신창동 100,257.0
26,서빙고공원,서울특별시 용산구 용산동5가 24-2,11879.0


In [4]:
park = park[park.소재지지번주소.str.contains('경기도')]
park.head()

,공원명,소재지지번주소,공원면적
212,꿈나무공원,경기도 이천시 중리동 15,2526.0
213,남천공원,경기도 이천시 중리동 210,3830.0
214,향교공원,경기도 이천시 창전동 408,1757.0
215,개나리공원,경기도 이천시 창전동 414-13,1650.0
216,달맞이공원,경기도 이천시 창전동 423,1650.0


In [6]:
park['시군구'] = park.소재지지번주소.apply(lambda x: 
                    x.split()[1] if x.split()[2][-1] != '구' else ' '.join(x.split()[1:3]))
park.head(100)

,공원명,소재지지번주소,공원면적,시군구
212,꿈나무공원,경기도 이천시 중리동 15,2526.0,이천시
213,남천공원,경기도 이천시 중리동 210,3830.0,이천시
214,향교공원,경기도 이천시 창전동 408,1757.0,이천시
215,개나리공원,경기도 이천시 창전동 414-13,1650.0,이천시
216,달맞이공원,경기도 이천시 창전동 423,1650.0,이천시
...,...,...,...,...
477,소공원,경기도 용인시 처인구 백암면 근창리 101-8,807.0,용인시 처인구
478,소공원,경기도 용인시 처인구 역북동 385-5,391.0,용인시 처인구
479,소공원,경기도 용인시 처인구 역북동 386-3,406.0,용인시 처인구
480,제99호 어린이공원,경기도 용인시 처인구 포곡읍 삼계리 671-25,1875.0,용인시 처인구


In [9]:
df = park.pivot_table('공원면적', '시군구', aggfunc='sum')
df.head()

,공원면적
시군구,
가평군,511674.5
고양시 덕양구,3287291.4
고양시 일산동구,2722980.0
고양시 일산서구,922486.9
과천시,315666.0


In [10]:
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop.head()

,행정구역구분명,행정구역명,총 인구수
0,도,경기도,13589432
1,시군,경기도 가평군,62197
2,읍면동,경기도 가평군 가평읍,19555
3,읍면동,경기도 가평군 북면,3815
4,읍면동,경기도 가평군 상면,5669


In [13]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
8,시군,경기도 고양시,1075202
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738


In [14]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [15]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ' '.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시 덕양구
31,구,경기도 고양시 일산동구,296590,고양시 일산동구
44,구,경기도 고양시 일산서구,290738,고양시 일산서구
56,시군,경기도 과천시,77775,과천시


In [16]:
df = df.merge(pop, left_index=True, right_on='시군구')
df['인당 공원면적'] = (df.공원면적 / df['총 인구수']).round(1)
df.head()

,공원면적,행정구역구분명,행정구역명,총 인구수,시군구,인당 공원면적
1,511674.5,시군,경기도 가평군,62197,가평군,8.2
9,3287291.4,구,경기도 고양시 덕양구,487874,고양시 덕양구,6.7
31,2722980.0,구,경기도 고양시 일산동구,296590,고양시 일산동구,9.2
44,922486.9,구,경기도 고양시 일산서구,290738,고양시 일산서구,3.2
56,315666.0,시군,경기도 과천시,77775,과천시,4.1
